In [1]:
import keras
import tensorflow as tf
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, BatchNormalization, Conv2D, Dropout, Activation, Flatten, MaxPooling2D
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
from keras import regularizers, optimizers
from tensorflow.keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [3]:
(x_train, x_valid) = x_train[5000:], x_train[:5000]
(y_train, y_valid) = y_train[5000:], y_train[:5000]

In [4]:
mean = np.mean(x_train, axis=(0, 1, 2, 3))
std = np.std(x_train, axis=(0, 1, 2, 3))
x_train = (x_train - mean)/std
x_test = (x_test - mean)/std
x_vaild = (x_valid - mean)/std

In [5]:
n_classes = 10
y_train = to_categorical(y_train, n_classes)
y_test = to_categorical(y_test, n_classes)
y_valid = to_categorical(y_valid, n_classes)

In [6]:
datagen = ImageDataGenerator(rotation_range=15,
                            width_shift_range=0.1,
                            height_shift_range=0.1,
                            horizontal_flip=True,
                            vertical_flip=False)

In [34]:
def inception(previous_layer, kernel_size_1= (1, 1), kernel_size_2= (1, 1), kernel_size_3= (3, 3),
                   kernel_size_4= (5, 5), kernel_size_5= (1, 1), kernel_size_6= (1, 1), max_pool= (3, 3), 
                   n_filters_1= None, n_filters_2=None, n_filters_3=None, 
                   n_filters_4=None, n_filters_5=None, n_filters_6=None):
    
    x1 = keras.layers.Conv2D(filters=n_filters_1, kernel_size=kernel_size_1)(previous_layer)
    x2 = keras.layers.Conv2D(filters=n_filters_2, kernel_size=kernel_size_2)(previous_layer)
    x3 = keras.layers.MaxPool2D()(previous_layer)
    x7 = keras.layers.Conv2D(filters=n_filters_6, kernel_size=kernel_size_6)(previous_layer)
    x5 = keras.layers.Conv2D(filters=n_filters_3, kernel_size=kernel_size_4)(x1)
    x6 = keras.layers.Conv2D(filters=n_filters_4, kernel_size=kernel_size_4)(x2)
    x7 = keras.layers.Conv2D(filters=n_filters_5, kernel_size=kernel_size_5)(x3)
    
    y = keras.layers.Concatenate(axis=3)([x4, x5, x6, x7])
    return y

In [35]:
def model(input_shape=(224, 224, 3)):
    # the most bottom module
    inputs = keras.Input(shape=input_shape)
    x = keras.layers.Conv2D(64, (7, 7), padding='same', activation='relu', strides=(2, 2))(inputs)
    x = keras.layers.MaxPool2D(pool_size=(3, 2))(x)
    x = keras.layers.MaxPool2D((3, 3), padding='same', strides=(2, 2))(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Conv2D(64, (1, 1), padding='same', strides=(1, 1), activation='relu')(x)
    x = keras.layers.Conv2D(192, (3, 3), padding='same', strides=(1, 1), activation='relu')(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.MaxPool2D((3, 3), padding='same', strides=(2, 2))(x)
    # first inception block consists of 2 modules
    x = inception(x, n_filters_1=96 , n_filters_2=16, n_filters_3=32, n_filters_4=32, n_filters_5=64, n_filters_6=128)
    x = inception(x, n_filters_1=128 , n_filters_2=32, n_filters_3=96, n_filters_4=64, n_filters_5=128, n_filters_6=192)
    
    x = keras.layers.MaxPool2D((3, 3), padding='same', strides=(2, 2))(x)
    
    x = inception(x, n_filters_1=96 , n_filters_2=16, n_filters_3=48, n_filters_4=32, n_filters_5=192, n_filters_6=208)
    x = inception(x, n_filters_1=112 , n_filters_2=24, n_filters_3=64, n_filters_4=64, n_filters_5=160, n_filters_6=224)
    x = inception(x, n_filters_1=128 , n_filters_2=24, n_filters_3=64, n_filters_4=64, n_filters_5=128, n_filters_6=256)
    x = inception(x, n_filters_1=144 , n_filters_2=32, n_filters_3=64, n_filters_4=64, n_filters_5=112, n_filters_6=288)
    x = inception(x, n_filters_1=160 , n_filters_2=32, n_filters_3=128, n_filters_4=128, n_filters_5=256, n_filters_6=320)
    
    x = keras.layers.MaxPool2D((3, 3), padding='same', strides=(2, 2))(x)
    x = inception(x, n_filters_1=160 , n_filters_2=32, n_filters_3=128, n_filters_4=128, n_filters_5=256, n_filters_6=320)
    x = inception(x, n_filters_1=192 , n_filters_2=48, n_filters_3=128, n_filters_4=128, n_filters_5=384, n_filters_6=384)
    
    x = keras.layers.AveragePooling2D(pool_size=(7, 7), strides=1, padding='valid')(x)
    x = keras.layers.Dropout(0.4)(x)
    x = keras.layers.Dense(10, activation='softmax')(x)
    
    model = keras.Model(inputs=inputs, outputs=x)
    return model

In [36]:
model = model()

ValueError: A `Concatenate` layer requires inputs with matching shapes except for the concatenation axis. Received: input_shape=[(None, 10, 14, 128), (None, 8, 12, 32), (None, 6, 10, 32), (None, 5, 7, 64)]